# Vortrainiertes CNN zur Erkennung von Autos

Dieses Notebook implementiert ein vortrainiertes CNN (MobileNetV2) mit Transfer Learning zur Erkennung von Autos im CIFAR-10 Datensatz.

## Überblick
- Laden eines vortrainierten MobileNetV2-Modells
- Anpassung des Modells für die Autoerkennung durch Transfer Learning
- Training des angepassten Modells
- Evaluierung des Modells auf Testdaten
- Visualisierung der Ergebnisse

## Importieren der benötigten Bibliotheken

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

## Vorbereitung der Verzeichnisse

Wir erstellen ein Verzeichnis für die Modelle, falls es noch nicht existiert.

In [ ]:
# Verzeichnisse erstellen
models_dir = os.path.join('..', 'models')
os.makedirs(models_dir, exist_ok=True)

## Laden der vorbereiteten Daten

Wir laden die im ersten Notebook vorbereiteten Daten.

In [ ]:
# Laden der vorbereiteten Daten
print("Laden der vorbereiteten Daten...")
x_train = np.load(os.path.join('..', 'data', 'x_train.npy'))
y_train = np.load(os.path.join('..', 'data', 'y_train.npy'))
x_test = np.load(os.path.join('..', 'data', 'x_test.npy'))
y_test = np.load(os.path.join('..', 'data', 'y_test.npy'))

print(f"Trainingsdaten: {len(x_train)} Bilder")
print(f"Testdaten: {len(x_test)} Bilder")

## Vorbereitung der Bilder für das vortrainierte Modell

Das vortrainierte MobileNetV2-Modell erwartet Bilder mit einer Größe von 224x224 Pixeln. Wir müssen daher die CIFAR-10 Bilder (32x32 Pixel) auf diese Größe skalieren.

In [ ]:
# Bilder auf die Größe anpassen, die das vortrainierte Modell erwartet
input_shape = (224, 224, 3)
x_train_resized = tf.image.resize(x_train, (input_shape[0], input_shape[1]))
x_test_resized = tf.image.resize(x_test, (input_shape[0], input_shape[1]))

## Laden des vortrainierten Modells

Wir laden das vortrainierte MobileNetV2-Modell, das auf dem ImageNet-Datensatz trainiert wurde. Wir entfernen die oberen Schichten (include_top=False), da wir unsere eigenen Schichten für die Autoerkennung hinzufügen werden.

In [ ]:
# Vortrainiertes Modell laden
print("Laden des vortrainierten MobileNetV2-Modells...")
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

## Einfrieren der vortrainierten Schichten

Wir frieren die Gewichte der vortrainierten Schichten ein, damit sie während des Trainings nicht aktualisiert werden. Dies ist ein wichtiger Schritt beim Transfer Learning, da wir die bereits gelernten Features beibehalten möchten.

In [ ]:
# Einfrieren der vortrainierten Schichten
for layer in base_model.layers:
    layer.trainable = False

## Hinzufügen von benutzerdefinierten Schichten

Wir fügen benutzerdefinierte Schichten hinzu, um das Modell für unsere spezifische Aufgabe (Autoerkennung) anzupassen:
1. GlobalAveragePooling2D: Reduziert die räumlichen Dimensionen
2. Dense Layer mit 1024 Neuronen und ReLU-Aktivierung
3. Output Layer mit einem Neuron und Sigmoid-Aktivierung für binäre Klassifikation

In [ ]:
# Hinzufügen von benutzerdefinierten Schichten für die Autoerkennung
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Erstellen des Modells
model = Model(inputs=base_model.input, outputs=predictions)

## Kompilieren des Modells

Wir kompilieren das Modell mit dem Adam-Optimizer, der binären Kreuzentropie als Verlustfunktion und der Genauigkeit als Metrik.

In [ ]:
# Kompilieren des Modells
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Zusammenfassung des Modells anzeigen
model.summary()

## Callbacks für das Training

Wir definieren zwei Callbacks für das Training:
1. ModelCheckpoint: Speichert das beste Modell basierend auf der Validierungsgenauigkeit
2. EarlyStopping: Beendet das Training, wenn sich die Validierungsgenauigkeit für 5 Epochen nicht verbessert

In [ ]:
# Callbacks für das Training
checkpoint = ModelCheckpoint(
    os.path.join(models_dir, 'pretrained_car_detection_model.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max',
    verbose=1
)

## Training des Modells

Wir trainieren das Modell mit den folgenden Parametern:
- Maximale Anzahl an Epochen: 20
- Batch-Größe: 32
- Validierungsdaten: Testdatensatz
- Callbacks: ModelCheckpoint und EarlyStopping

In [ ]:
# Training des Modells
print("Training des vortrainierten Modells...")
history = model.fit(
    x_train_resized, y_train,
    validation_data=(x_test_resized, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint, early_stopping]
)

## Evaluierung des Modells

Wir evaluieren das trainierte Modell auf den Testdaten.

In [ ]:
# Evaluierung des Modells
print("Evaluierung des Modells auf den Testdaten...")
test_loss, test_accuracy = model.evaluate(x_test_resized, y_test)
print(f"Testgenauigkeit: {test_accuracy:.4f}")

## Visualisierung der Trainingsergebnisse

Wir visualisieren den Verlauf der Genauigkeit und des Verlusts während des Trainings.

In [ ]:
# Visualisierung der Trainingsergebnisse
plt.figure(figsize=(12, 4))

# Genauigkeit
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Modellgenauigkeit')
plt.ylabel('Genauigkeit')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validierung'], loc='lower right')

# Verlust
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Modellverlust')
plt.ylabel('Verlust')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validierung'], loc='upper right')

plt.tight_layout()
plt.savefig(os.path.join(models_dir, 'pretrained_model_training.png'))
print(f"Trainingsverlauf wurde gespeichert unter: {os.path.join(models_dir, 'pretrained_model_training.png')}")

## Speichern des Modells

Wir speichern das trainierte Modell für die spätere Verwendung.

In [ ]:
# Speichern des Modells
model.save(os.path.join(models_dir, 'pretrained_car_detection_model.keras'))
print(f"Modell wurde gespeichert unter: {os.path.join(models_dir, 'pretrained_car_detection_model.keras')}")

## Visualisierung der Vorhersagen

Wir visualisieren einige Vorhersagen des Modells auf den Testdaten.

In [ ]:
# Funktion zur Visualisierung der Vorhersagen
def plot_predictions(X, y, predictions, num_images=25):
    plt.figure(figsize=(10, 10))
    for i in range(min(num_images, len(X))):
        plt.subplot(5, 5, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X[i])
        
        color = 'green' if predictions[i] == y[i] else 'red'
        label = "Auto" if predictions[i] == 1 else "Kein Auto"
        plt.xlabel(label, color=color)
    
    plt.tight_layout()
    plt.savefig(os.path.join(models_dir, 'pretrained_model_predictions.png'))
    print(f"Vorhersagen wurden gespeichert unter: {os.path.join(models_dir, 'pretrained_model_predictions.png')}")

# Vorhersagen für Testdaten
print("Generieren von Vorhersagen für Testdaten...")
predictions = (model.predict(x_test_resized) > 0.5).astype(int).flatten()
plot_predictions(x_test, y_test, predictions)

## Zusammenfassung

In diesem Notebook haben wir:
1. Ein vortrainiertes MobileNetV2-Modell geladen
2. Das Modell für die Autoerkennung angepasst durch Transfer Learning
3. Das angepasste Modell trainiert
4. Das Modell auf Testdaten evaluiert
5. Die Ergebnisse visualisiert

Transfer Learning ist eine leistungsstarke Technik, die es uns ermöglicht, von vortrainierten Modellen zu profitieren und sie für spezifische Aufgaben anzupassen. Dies spart Rechenressourcen und Zeit, da wir nicht von Grund auf trainieren müssen.

In [ ]:
print("Training und Evaluierung des vortrainierten Modells abgeschlossen.")